In [7]:
def cnn_scrap(keyword):
    import webbrowser
    import requests
    from bs4 import BeautifulSoup
    import os
    import re
    from selenium import webdriver
    chrome_path=r"C:\Users\ADMIN\Desktop\CIMB Work\chromedriver.exe"
    from datetime import datetime, timedelta
    import warnings
    warnings.filterwarnings('ignore')
    from fuzzywuzzy import fuzz
    import pandas as pd
    import numpy as np
    import time
    
    todaysdate=datetime.now().strftime("%Y-%m-%d")
    last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
    url_base=[]
    headline=[]
    headline_url=[]
    cat=[]
    date=[]
    article=[]
    
    for key in keyword:
        base_url="https://edition.cnn.com/search?size=20&q={}&sort=newest".format(key)
        driver=webdriver.Chrome(chrome_path)
        driver.get(base_url)
        time.sleep(3)
        real_soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        ab=real_soup.find_all("div", {"class":"cnn-search__result cnn-search__result--article"})
        if(ab!=None):
            for i in ab:
                link=i.find("a",href=True)
                link1=link['href'].strip().replace("//",'')
                headline_url.append(str(link1))
                headline_1 = i.find("h3").get_text().strip()
                headline.append(str(headline_1))
                url_base.append(base_url)
                cat.append(key)
                date_1=i.find('div',attrs={'class':'cnn-search__result-publish-date'}).get_text().strip()
                date.append(str(date_1))
                article_1=i.find('div',attrs={'class':'cnn-search__result-body'}).get_text().strip()
                article.append(article_1)
        else:
            headline_url.append("No url found")
            headline.append("No headline found")
            url_base.append(base_url)
            cat.append(key)
            date.append("")
            article.append("")
        

    df = pd.DataFrame({'keyword':cat,'main_url':url_base, 'Headline':headline,
                       'headline_url': headline_url, 'Date':date,'article':article })
    df.drop_duplicates(inplace=True)
    df['website']="CNN"

    df['Match'] = df.apply(lambda x: fuzz.partial_ratio(x['keyword'], x['article']), axis=1)
    df['Match_1'] = df.apply(lambda x: fuzz.token_set_ratio(x['keyword'], x['article']), axis=1)
    
    return df

In [8]:
cnn_df=cnn_scrap(keyword)

In [10]:
cnn_df

,keyword,main_url,Headline,headline_url,Date,article,website,Match,Match_1
0,RP TRADING,https://edition.cnn.com/search?size=20&q=RP TR...,EU fines banks record $2.3B over Libor,http:money.cnn.com/2013/12/04/news/companies/l...,"Dec 4, 2013",The European Union has levied a record antitru...,CNN,30,100
1,RP TRADING,https://edition.cnn.com/search?size=20&q=RP TR...,Three ex-ICAP employees indicted in Libor scandal,http:money.cnn.com/2013/09/25/news/companies/i...,"Sep 25, 2013",The Justice Department announced criminal char...,CNN,30,100
2,RP TRADING,https://edition.cnn.com/search?size=20&q=RP TR...,Rate-fixing scandal shakes three continents,www.cnn.com/2013/02/06/business/libor-rate-fix...,"Feb 7, 2013",The fallout from the interest rate manipulatio...,CNN,30,100


In [11]:
cnn_df['Date']

0     Dec 4, 2013
1    Sep 25, 2013
2     Feb 7, 2013
Name: Date, dtype: object

In [12]:
import time
from datetime import datetime, timedelta
todaysdate=datetime.now().strftime("%Y-%m-%d")
last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")

In [17]:
cnn_df['date_time_1'] = cnn_df["Date"].apply(lambda x : datetime.strptime(str(x), '%b %d, %Y') if x!='' else None)

In [18]:
# print(datetime.strptime(str(cnn_df['Date'][1]), '%b %d, %Y'))
cnn_df['date_time_1']

0   2013-12-04
1   2013-09-25
2   2013-02-07
Name: date_time_1, dtype: datetime64[ns]

In [32]:
type(todaysdate)

datetime.datetime

In [35]:
import time
from datetime import datetime, timedelta
todaysdate=datetime.now().strftime("%Y-%m-%d")
# todaysdate=datetime.strptime(str(todaysdate), '%Y-%m-%d')
last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
# last_date=datetime.strptime(str(last_date), '%Y-%m-%d')
cnn_df['date_time_1']


0   2013-12-04
1   2013-09-25
2   2013-02-07
Name: date_time_1, dtype: datetime64[ns]

In [36]:
import numpy as np
cnn_df['date_time_1'] = cnn_df["Date"].apply(lambda x : datetime.strptime(str(x), '%b %d, %Y') if x!='' else None)
cnn_df['trigger']=np.where(((cnn_df['date_time_1']>=last_date)
                                 &(cnn_df['date_time_1']<=todaysdate))
                                & (cnn_df['Match_1']>90),'Y','N')

In [37]:
cnn_df['trigger']

0    N
1    N
2    N
Name: trigger, dtype: object